At the moment I have narrowed it down to 22 features which have >0.5 correlation with the target and are of general importance. Now I need to normalize the continous variables so that everything is on the same scale and add 13 more features to get our 35 Total. At the end I show which features I drop and which I Still have left. Lots of work to do...

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import pointbiserialr

In [3]:
DF = pd.read_csv('./train.csv')
DF.shape
aList = list(DF.dtypes)
categorical_columns = list( DF.select_dtypes(include=['object']).columns )
numerical_columns = list( DF.select_dtypes(include=['int64']).columns )
print("Number of continous features", len(numerical_columns))
print("Number of Categorical features", len(categorical_columns))
print("Total Features", len(list(DF.columns)))
target=DF['SalePrice']
droppedCols = []

Number of continous features 36
Number of Categorical features 43
Total Features 82


Approach: Examine Categorical Features for
<br> 
-correlation to target: If the feature is not highly correlated consider dropping, Considering 0.4-0.5 Moderate and >0.5 strong correlation
<br> 
-Multicollinearity: if the feature is correllated with another feature consider keeping one or the other
<br> 
-General Importance: if the feature seems unimportant to our target consider dropping
<br> 
-Missing values: if the feature has over 50% missing values consider dropping. 
<br> 
-One Hot Encode Categorical variables to numeric ones. Ie create a new column and convert the results to binary values. 



Approach For Continous Features 
-correlation to target
-multicollinearity
-general importance
Normalize values, this will help with algorithms like kmeans and gradient descent(still need to do)

In [4]:
def check_correlation(data, target_variable):
    """
    Check the correlation of continous features with the target variable.

    Parameters:
    - data: DataFrame, the dataset containing features and the target variable
    - target_variable: str, the name of the target variable

    Returns:
    - correlation_df: DataFrame, a DataFrame showing the correlation of each feature with the target variable
    """
    correlation_df = data.corr()[[target_variable]]
    correlation_df = correlation_df.sort_values(by=target_variable, ascending=False)
    return correlation_df
def check_categorical_correlation(data, feature, target_variable):
    """
    Check the correlation of a categorical feature with the target variable.

    Parameters:
    - data: DataFrame, the dataset containing the categorical feature and the target variable
    - feature: str, the name of the categorical feature
    - target_variable: str, the name of the target variable

    Returns:
    - correlation_result: float, the correlation of the specified categorical feature with the target variable
    """
    # Calculate correlation for the specified categorical feature using point-biserial correlation
    correlation_result, _ = pointbiserialr(data[feature], data[target_variable])
    return correlation_result
def inspectCol(column, data):
    print("Inspecting Column:", column)
    print("Data Type:", data[column].dtype)
    print("Number of Unique Categories:", len(data[column].unique()))
    print("Unique Categories:", data[column].unique())
    print("Missing Values:", data[column].isnull().sum())
    print()
def percentMissing(col, DF):
    missingRate = DF[col].isnull().sum()/len(DF)
    print(f'Missing rate of {col}: {missingRate}')
def one_hot_encode(column, data):
    one_hot_encoded = pd.get_dummies(data[column], prefix=column, drop_first=True)
    return one_hot_encoded
def updateDataFrame(DF, drop):
    if drop != None:
    # Drop the specified columns from the original DataFrame in place
        DF.drop(drop, axis=1, inplace=True)

    # Display information about the updated DataFrame
    categorical_columns = list(DF.select_dtypes(include=['object']).columns)
    numerical_columns = list(DF.select_dtypes(include=['int64']).columns)
    
    print("Number of continuous features:", len(numerical_columns))
    print("Number of categorical features:", len(categorical_columns))
    print("Total Features:", len(list(DF.columns)))

    return DF, categorical_columns, numerical_columns
def process_categorical_column(df, categorical_column, target_variable, correlation_threshold=0.5):
    # One-hot encode the categorical column
    one_hot_encoded_df = one_hot_encode(categorical_column, df)
    print("One-Hot Encoded DataFrame:")
    print(one_hot_encoded_df)

    # Concatenate the one-hot encoded DataFrame with the original DataFrame
    df_with_one_hot = pd.concat([df, one_hot_encoded_df], axis=1)

    # Check correlation with the target variable and drop if necessary
    cols = list(one_hot_encoded_df.columns)
    for col in cols:
        correlation_result = check_categorical_correlation(df_with_one_hot, col, target_variable)
        print(f"{col} Correlation with {target_variable}: {correlation_result}")

    return df_with_one_hot
def add_encoded_columns(original_df, encoded_df, columns_to_add):
    """
    Add specific one-hot encoded columns to the original DataFrame.

    Parameters:
    - original_df: DataFrame, the original DataFrame
    - encoded_df: DataFrame, the DataFrame with one-hot encoded columns
    - columns_to_add: list, a list of column names to add to the original DataFrame

    Returns:
    - updated_df: DataFrame, the updated DataFrame with added columns
    """
    # Ensure columns_to_add is a list even if it contains a single column
    if not isinstance(columns_to_add, list):
        columns_to_add = [columns_to_add]

    # Select the desired columns from the one-hot encoded DataFrame
    selected_columns = encoded_df[columns_to_add]

    # Concatenate the selected columns with the original DataFrame
    updated_df = pd.concat([original_df, selected_columns], axis=1)

    return updated_df

# 1
MSZoning is the first, RL meaning Residential Low density,etc.
Nominal Data should be converted into corresponding strings

In [5]:
inspectCol(categorical_columns[0], DF)
DF_with_processed_categorical = process_categorical_column(DF, categorical_columns[0], target_variable='SalePrice')

Inspecting Column: MSZoning
Data Type: object
Number of Unique Categories: 5
Unique Categories: ['RL' 'FV' 'RM' 'C (all)' 'RH']
Missing Values: 0

One-Hot Encoded DataFrame:
      MSZoning_FV  MSZoning_RH  MSZoning_RL  MSZoning_RM
0           False        False         True        False
1           False        False         True        False
2           False        False         True        False
3            True        False        False        False
4           False        False         True        False
...           ...          ...          ...          ...
1017        False        False         True        False
1018        False        False         True        False
1019        False        False         True        False
1020        False        False         True        False
1021        False        False         True        False

[1022 rows x 4 columns]
MSZoning_FV Correlation with SalePrice: 0.10405808377927131
MSZoning_RH Correlation with SalePrice: -0.07391227514199

Since all of the cols have low correlation with the target, going to drop it.

In [6]:
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'MSZoning')
droppedCols.append('MSZoning')


Number of continuous features: 36
Number of categorical features: 42
Total Features: 81


# 2
Street

In [7]:
inspectCol(categorical_columns[0], DF)
DF_with_processed_categorical = process_categorical_column(DF, categorical_columns[0], target_variable='SalePrice')

Inspecting Column: Street
Data Type: object
Number of Unique Categories: 2
Unique Categories: ['Pave' 'Grvl']
Missing Values: 0

One-Hot Encoded DataFrame:
      Street_Pave
0            True
1            True
2            True
3            True
4            True
...           ...
1017         True
1018         True
1019         True
1020         True
1021         True

[1022 rows x 1 columns]
Street_Pave Correlation with SalePrice: 0.03557372119219705


Low correlation with target varable so drop

In [8]:
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'Street')
droppedCols.append('Street')

Number of continuous features: 36
Number of categorical features: 41
Total Features: 80


# 3

In [9]:
inspectCol(categorical_columns[0], DF)
percentMissing(categorical_columns[0], DF)

Inspecting Column: Alley
Data Type: object
Number of Unique Categories: 3
Unique Categories: [nan 'Pave' 'Grvl']
Missing Values: 962

Missing rate of Alley: 0.9412915851272016


Over 90% missing values, going to assume that we can drop this one

In [10]:
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'Alley')
droppedCols.append('Alley')

Number of continuous features: 36
Number of categorical features: 40
Total Features: 79


# 4

In [11]:
inspectCol(categorical_columns[0], DF)
DF_with_processed_categorical = process_categorical_column(DF, categorical_columns[0], target_variable='SalePrice')

Inspecting Column: LotShape
Data Type: object
Number of Unique Categories: 4
Unique Categories: ['Reg' 'IR1' 'IR2' 'IR3']
Missing Values: 0

One-Hot Encoded DataFrame:
      LotShape_IR2  LotShape_IR3  LotShape_Reg
0            False         False          True
1            False         False          True
2            False         False          True
3            False         False         False
4            False         False         False
...            ...           ...           ...
1017         False         False          True
1018         False         False          True
1019         False         False          True
1020         False         False          True
1021         False         False         False

[1022 rows x 3 columns]
LotShape_IR2 Correlation with SalePrice: 0.11525415119840204
LotShape_IR3 Correlation with SalePrice: 0.03719930320082107
LotShape_Reg Correlation with SalePrice: -0.26136652224636314


Weak correlation so drop

In [12]:
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'LotShape')
droppedCols.append('LotShape')

Number of continuous features: 36
Number of categorical features: 39
Total Features: 78


# 5

In [13]:
inspectCol(categorical_columns[0], DF)
DF_with_processed_categorical = process_categorical_column(DF, categorical_columns[0], target_variable='SalePrice')

Inspecting Column: LandContour
Data Type: object
Number of Unique Categories: 4
Unique Categories: ['Lvl' 'Low' 'Bnk' 'HLS']
Missing Values: 0

One-Hot Encoded DataFrame:
      LandContour_HLS  LandContour_Low  LandContour_Lvl
0               False            False             True
1               False            False             True
2               False            False             True
3               False            False             True
4               False            False             True
...               ...              ...              ...
1017            False            False             True
1018            False            False             True
1019            False            False             True
1020            False            False            False
1021            False            False             True

[1022 rows x 3 columns]
LandContour_HLS Correlation with SalePrice: 0.13729530290488345
LandContour_Low Correlation with SalePrice: 0.04446641044326761
Land

Weak correlation so drop

In [14]:
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'LandContour')
droppedCols.append('LandContour')

Number of continuous features: 36
Number of categorical features: 38
Total Features: 77


# 6

In [15]:
inspectCol(categorical_columns[0], DF)
DF_with_processed_categorical = process_categorical_column(DF, categorical_columns[0], target_variable='SalePrice')

Inspecting Column: Utilities
Data Type: object
Number of Unique Categories: 2
Unique Categories: ['AllPub' 'NoSeWa']
Missing Values: 0

One-Hot Encoded DataFrame:
      Utilities_NoSeWa
0                False
1                False
2                False
3                False
4                False
...                ...
1017             False
1018             False
1019             False
1020             False
1021             False

[1022 rows x 1 columns]
Utilities_NoSeWa Correlation with SalePrice: -0.016854843033484397


Weak Correlation so drop

In [16]:
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'Utilities')
droppedCols.append('Utilities')

Number of continuous features: 36
Number of categorical features: 37
Total Features: 76


# 7

In [17]:
inspectCol(categorical_columns[0], DF)
DF_with_processed_categorical = process_categorical_column(DF, categorical_columns[0], target_variable='SalePrice')

Inspecting Column: LotConfig
Data Type: object
Number of Unique Categories: 5
Unique Categories: ['Inside' 'Corner' 'CulDSac' 'FR2' 'FR3']
Missing Values: 0

One-Hot Encoded DataFrame:
      LotConfig_CulDSac  LotConfig_FR2  LotConfig_FR3  LotConfig_Inside
0                 False          False          False              True
1                 False          False          False              True
2                 False          False          False             False
3                 False          False          False              True
4                 False          False          False             False
...                 ...            ...            ...               ...
1017              False          False          False             False
1018              False          False          False              True
1019              False          False          False              True
1020              False          False          False             False
1021              False

weak correlation so drop

In [18]:
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'LotConfig')
droppedCols.append('LotConfig')

Number of continuous features: 36
Number of categorical features: 36
Total Features: 75


# 8

In [19]:
inspectCol(categorical_columns[0], DF)
DF_with_processed_categorical = process_categorical_column(DF, categorical_columns[0], target_variable='SalePrice')

Inspecting Column: LandSlope
Data Type: object
Number of Unique Categories: 3
Unique Categories: ['Gtl' 'Mod' 'Sev']
Missing Values: 0

One-Hot Encoded DataFrame:
      LandSlope_Mod  LandSlope_Sev
0             False          False
1             False          False
2             False          False
3             False          False
4             False          False
...             ...            ...
1017          False          False
1018          False          False
1019          False          False
1020           True          False
1021          False          False

[1022 rows x 2 columns]
LandSlope_Mod Correlation with SalePrice: 0.03159273420825288
LandSlope_Sev Correlation with SalePrice: 0.03860688219284656


weak correlation so drop

In [20]:
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'LandSlope')
droppedCols.append('LandSlope')

Number of continuous features: 36
Number of categorical features: 35
Total Features: 74


# 9

In [21]:
inspectCol(categorical_columns[0], DF)
DF_with_processed_categorical = process_categorical_column(DF, categorical_columns[0], target_variable='SalePrice')

Inspecting Column: Neighborhood
Data Type: object
Number of Unique Categories: 24
Unique Categories: ['CollgCr' 'Edwards' 'NAmes' 'Somerst' 'Timber' 'Veenker' 'OldTown'
 'BrkSide' 'ClearCr' 'MeadowV' 'Mitchel' 'SawyerW' 'StoneBr' 'NoRidge'
 'SWISU' 'Sawyer' 'Gilbert' 'IDOTRR' 'NridgHt' 'Blmngtn' 'NWAmes'
 'Crawfor' 'BrDale' 'NPkVill']
Missing Values: 0

One-Hot Encoded DataFrame:
      Neighborhood_BrDale  Neighborhood_BrkSide  Neighborhood_ClearCr  \
0                   False                 False                 False   
1                   False                 False                 False   
2                   False                 False                 False   
3                   False                 False                 False   
4                   False                 False                 False   
...                   ...                   ...                   ...   
1017                False                 False                 False   
1018                False        

weak correlation so drop

In [22]:
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'Neighborhood')
droppedCols.append('Neighborhood')

Number of continuous features: 36
Number of categorical features: 34
Total Features: 73


# 10

In [23]:
inspectCol(categorical_columns[0], DF)
DF_with_processed_categorical = process_categorical_column(DF, categorical_columns[0], target_variable='SalePrice')

Inspecting Column: Condition1
Data Type: object
Number of Unique Categories: 9
Unique Categories: ['Norm' 'Artery' 'RRAn' 'Feedr' 'RRNn' 'PosN' 'RRAe' 'PosA' 'RRNe']
Missing Values: 0

One-Hot Encoded DataFrame:
      Condition1_Feedr  Condition1_Norm  Condition1_PosA  Condition1_PosN  \
0                False             True            False            False   
1                False            False            False            False   
2                False            False            False            False   
3                False             True            False            False   
4                False             True            False            False   
...                ...              ...              ...              ...   
1017             False             True            False            False   
1018             False             True            False            False   
1019             False             True            False            False   
1020             F

Weak correlation so drop

In [24]:
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'Condition1')
droppedCols.append('Condition1')

Number of continuous features: 36
Number of categorical features: 33
Total Features: 72


# 11

In [25]:
inspectCol(categorical_columns[0], DF)
DF_with_processed_categorical = process_categorical_column(DF, categorical_columns[0], target_variable='SalePrice')

Inspecting Column: Condition2
Data Type: object
Number of Unique Categories: 8
Unique Categories: ['Norm' 'RRAe' 'Feedr' 'Artery' 'PosA' 'RRNn' 'PosN' 'RRAn']
Missing Values: 0

One-Hot Encoded DataFrame:
      Condition2_Feedr  Condition2_Norm  Condition2_PosA  Condition2_PosN  \
0                False             True            False            False   
1                False             True            False            False   
2                False             True            False            False   
3                False             True            False            False   
4                False             True            False            False   
...                ...              ...              ...              ...   
1017             False             True            False            False   
1018             False             True            False            False   
1019             False             True            False            False   
1020             False   

weak corr so drop

In [26]:
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'Condition2')
droppedCols.append('Condition2')

Number of continuous features: 36
Number of categorical features: 32
Total Features: 71


# 12

In [27]:
inspectCol(categorical_columns[0], DF)
DF_with_processed_categorical = process_categorical_column(DF, categorical_columns[0], target_variable='SalePrice')

Inspecting Column: BldgType
Data Type: object
Number of Unique Categories: 5
Unique Categories: ['1Fam' 'TwnhsE' 'Duplex' 'Twnhs' '2fmCon']
Missing Values: 0

One-Hot Encoded DataFrame:
      BldgType_2fmCon  BldgType_Duplex  BldgType_Twnhs  BldgType_TwnhsE
0               False            False           False            False
1               False            False           False            False
2               False            False           False            False
3               False            False           False             True
4               False            False           False            False
...               ...              ...             ...              ...
1017            False            False           False             True
1018            False            False           False            False
1019            False             True           False            False
1020            False            False           False            False
1021            False 

weak corr so drop

In [28]:
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'BldgType')
droppedCols.append('BldgType')

Number of continuous features: 36
Number of categorical features: 31
Total Features: 70


# 13

In [29]:
inspectCol(categorical_columns[0], DF)
DF_with_processed_categorical = process_categorical_column(DF, categorical_columns[0], target_variable='SalePrice')

Inspecting Column: HouseStyle
Data Type: object
Number of Unique Categories: 8
Unique Categories: ['2Story' 'SLvl' '1Story' '1.5Fin' 'SFoyer' '1.5Unf' '2.5Unf' '2.5Fin']
Missing Values: 0

One-Hot Encoded DataFrame:
      HouseStyle_1.5Unf  HouseStyle_1Story  HouseStyle_2.5Fin  \
0                 False              False              False   
1                 False              False              False   
2                 False              False              False   
3                 False               True              False   
4                 False               True              False   
...                 ...                ...                ...   
1017              False              False              False   
1018              False               True              False   
1019              False               True              False   
1020              False               True              False   
1021              False               True              False   

   

In [30]:
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'HouseStyle')
droppedCols.append('HouseStyle')

Number of continuous features: 36
Number of categorical features: 30
Total Features: 69


# 14

In [31]:
inspectCol(categorical_columns[0], DF)
DF_with_processed_categorical = process_categorical_column(DF, categorical_columns[0], target_variable='SalePrice')

Inspecting Column: RoofStyle
Data Type: object
Number of Unique Categories: 6
Unique Categories: ['Gable' 'Hip' 'Shed' 'Flat' 'Gambrel' 'Mansard']
Missing Values: 0

One-Hot Encoded DataFrame:
      RoofStyle_Gable  RoofStyle_Gambrel  RoofStyle_Hip  RoofStyle_Mansard  \
0                True              False          False              False   
1                True              False          False              False   
2                True              False          False              False   
3                True              False          False              False   
4                True              False          False              False   
...               ...                ...            ...                ...   
1017             True              False          False              False   
1018            False              False           True              False   
1019             True              False          False              False   
1020             True      

In [32]:
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'RoofStyle')
droppedCols.append('RoofStyle')

Number of continuous features: 36
Number of categorical features: 29
Total Features: 68


# 15

In [33]:
inspectCol(categorical_columns[0], DF)
DF_with_processed_categorical = process_categorical_column(DF, categorical_columns[0], target_variable='SalePrice')

Inspecting Column: RoofMatl
Data Type: object
Number of Unique Categories: 7
Unique Categories: ['CompShg' 'Metal' 'Tar&Grv' 'WdShake' 'Membran' 'WdShngl' 'ClyTile']
Missing Values: 0

One-Hot Encoded DataFrame:
      RoofMatl_CompShg  RoofMatl_Membran  RoofMatl_Metal  RoofMatl_Tar&Grv  \
0                 True             False           False             False   
1                 True             False           False             False   
2                 True             False           False             False   
3                 True             False           False             False   
4                 True             False           False             False   
...                ...               ...             ...               ...   
1017              True             False           False             False   
1018              True             False           False             False   
1019              True             False           False             False   
1020    

In [34]:
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'RoofMatl')
droppedCols.append('RoofMatl')

Number of continuous features: 36
Number of categorical features: 28
Total Features: 67


# 16

In [35]:
inspectCol(categorical_columns[0], DF)
DF_with_processed_categorical = process_categorical_column(DF, categorical_columns[0], target_variable='SalePrice')

Inspecting Column: Exterior1st
Data Type: object
Number of Unique Categories: 13
Unique Categories: ['VinylSd' 'MetalSd' 'Wd Sdng' 'WdShing' 'HdBoard' 'Plywood' 'AsphShn'
 'CemntBd' 'BrkFace' 'AsbShng' 'BrkComm' 'Stone' 'Stucco']
Missing Values: 0

One-Hot Encoded DataFrame:
      Exterior1st_AsphShn  Exterior1st_BrkComm  Exterior1st_BrkFace  \
0                   False                False                False   
1                   False                False                False   
2                   False                False                False   
3                   False                False                False   
4                   False                False                False   
...                   ...                  ...                  ...   
1017                False                False                False   
1018                False                False                False   
1019                False                False                False   
1020          

In [36]:
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'Exterior1st')
droppedCols.append('Exterior1st')

Number of continuous features: 36
Number of categorical features: 27
Total Features: 66


# 17

In [37]:
inspectCol(categorical_columns[0], DF)
DF_with_processed_categorical = process_categorical_column(DF, categorical_columns[0], target_variable='SalePrice')

Inspecting Column: Exterior2nd
Data Type: object
Number of Unique Categories: 15
Unique Categories: ['VinylSd' 'MetalSd' 'Wd Sdng' 'Wd Shng' 'HdBoard' 'Plywood' 'AsphShn'
 'CmentBd' 'BrkFace' 'Stone' 'AsbShng' 'Stucco' 'Brk Cmn' 'ImStucc'
 'Other']
Missing Values: 0

One-Hot Encoded DataFrame:
      Exterior2nd_AsphShn  Exterior2nd_Brk Cmn  Exterior2nd_BrkFace  \
0                   False                False                False   
1                   False                False                False   
2                   False                False                False   
3                   False                False                False   
4                   False                False                False   
...                   ...                  ...                  ...   
1017                False                 True                False   
1018                False                False                False   
1019                False                False                Fals

In [38]:
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'Exterior2nd')
droppedCols.append('Exterior2nd')

Number of continuous features: 36
Number of categorical features: 26
Total Features: 65


# 18

In [39]:
inspectCol(categorical_columns[0], DF)
percentMissing(categorical_columns[0], DF)
DF_with_processed_categorical = process_categorical_column(DF, categorical_columns[0], target_variable='SalePrice')

Inspecting Column: MasVnrType
Data Type: object
Number of Unique Categories: 4
Unique Categories: ['BrkFace' nan 'Stone' 'BrkCmn']
Missing Values: 614

Missing rate of MasVnrType: 0.6007827788649707
One-Hot Encoded DataFrame:
      MasVnrType_BrkFace  MasVnrType_Stone
0                   True             False
1                  False             False
2                   True             False
3                  False             False
4                  False             False
...                  ...               ...
1017               False             False
1018                True             False
1019               False             False
1020               False             False
1021               False             False

[1022 rows x 2 columns]
MasVnrType_BrkFace Correlation with SalePrice: 0.18892147435530043
MasVnrType_Stone Correlation with SalePrice: 0.3428427445112777


In [40]:
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'MasVnrType')
droppedCols.append('MasVnrType')

Number of continuous features: 36
Number of categorical features: 25
Total Features: 64


# 19

In [41]:
inspectCol(categorical_columns[0], DF)
DF_with_processed_categorical = process_categorical_column(DF, categorical_columns[0], target_variable='SalePrice')

Inspecting Column: ExterQual
Data Type: object
Number of Unique Categories: 4
Unique Categories: ['TA' 'Gd' 'Ex' 'Fa']
Missing Values: 0

One-Hot Encoded DataFrame:
      ExterQual_Fa  ExterQual_Gd  ExterQual_TA
0            False         False          True
1            False         False          True
2            False         False          True
3            False          True         False
4            False         False          True
...            ...           ...           ...
1017         False         False          True
1018         False         False          True
1019         False         False          True
1020         False         False          True
1021         False         False          True

[1022 rows x 3 columns]
ExterQual_Fa Correlation with SalePrice: -0.1278527185612707
ExterQual_Gd Correlation with SalePrice: 0.4222712640461679
ExterQual_TA Correlation with SalePrice: -0.5801338456840202


found one! add ExterQual_TA to the DF

In [42]:
DF['ExterQual_TA'] = DF_with_processed_categorical['ExterQual_TA'] #add the new col
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'ExterQual') #drop the old one 
droppedCols.append('ExterQual_Gd')
droppedCols.append('ExterQual_Fa')

Number of continuous features: 36
Number of categorical features: 24
Total Features: 64


# 20

In [43]:
inspectCol(categorical_columns[0], DF)
DF_with_processed_categorical = process_categorical_column(DF, categorical_columns[0], target_variable='SalePrice')

Inspecting Column: ExterCond
Data Type: object
Number of Unique Categories: 5
Unique Categories: ['TA' 'Gd' 'Fa' 'Ex' 'Po']
Missing Values: 0

One-Hot Encoded DataFrame:
      ExterCond_Fa  ExterCond_Gd  ExterCond_Po  ExterCond_TA
0            False         False         False          True
1            False         False         False          True
2            False         False         False          True
3            False         False         False          True
4            False         False         False          True
...            ...           ...           ...           ...
1017         False         False         False          True
1018         False         False         False          True
1019         False         False         False          True
1020         False         False         False          True
1021         False         False         False          True

[1022 rows x 4 columns]
ExterCond_Fa Correlation with SalePrice: -0.15190109870504123
ExterCond_G

In [44]:
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'ExterCond')
droppedCols.append('ExterCond')

Number of continuous features: 36
Number of categorical features: 23
Total Features: 63


# 21

In [45]:
inspectCol(categorical_columns[0], DF)
DF_with_processed_categorical = process_categorical_column(DF, categorical_columns[0], target_variable='SalePrice')

Inspecting Column: Foundation
Data Type: object
Number of Unique Categories: 6
Unique Categories: ['PConc' 'CBlock' 'BrkTil' 'Slab' 'Stone' 'Wood']
Missing Values: 0

One-Hot Encoded DataFrame:
      Foundation_CBlock  Foundation_PConc  Foundation_Slab  Foundation_Stone  \
0                 False              True            False             False   
1                  True             False            False             False   
2                  True             False            False             False   
3                 False              True            False             False   
4                  True             False            False             False   
...                 ...               ...              ...               ...   
1017               True             False            False             False   
1018              False              True            False             False   
1019              False             False             True             False   
1020  

In [46]:
DF['Foundation_PConc'] = DF_with_processed_categorical['Foundation_PConc']
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'Foundation') #drop the old one 
droppedCols.append('Foundation_CBlock')
droppedCols.append('Foundation_Slab')
droppedCols.append('Foundation_Stone')
droppedCols.append('Foundation_Wood')

Number of continuous features: 36
Number of categorical features: 22
Total Features: 63


# 22

Removing missing values because there are so few

In [47]:
inspectCol(categorical_columns[0], DF)
percentMissing(categorical_columns[0], DF)
DF = DF.dropna(subset=['BsmtQual'])
percentMissing(categorical_columns[0], DF)
DF_with_processed_categorical = process_categorical_column(DF, categorical_columns[0], target_variable='SalePrice')

Inspecting Column: BsmtQual
Data Type: object
Number of Unique Categories: 5
Unique Categories: ['Gd' 'TA' nan 'Ex' 'Fa']
Missing Values: 29

Missing rate of BsmtQual: 0.02837573385518591
Missing rate of BsmtQual: 0.0
One-Hot Encoded DataFrame:
      BsmtQual_Fa  BsmtQual_Gd  BsmtQual_TA
0           False         True        False
1           False        False         True
2           False        False         True
3           False         True        False
4           False        False         True
...           ...          ...          ...
1016        False        False         True
1017        False         True        False
1018        False        False         True
1020        False        False         True
1021        False        False         True

[993 rows x 3 columns]
BsmtQual_Fa Correlation with SalePrice: -0.14076227732592225
BsmtQual_Gd Correlation with SalePrice: 0.2167438282976727
BsmtQual_TA Correlation with SalePrice: -0.48842221206948677


In [48]:
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'BsmtQual')
droppedCols.append('BsmtQual')

Number of continuous features: 36
Number of categorical features: 21
Total Features: 62


# 23

In [49]:
inspectCol(categorical_columns[0], DF)
DF_with_processed_categorical = process_categorical_column(DF, categorical_columns[0], target_variable='SalePrice')

Inspecting Column: BsmtCond
Data Type: object
Number of Unique Categories: 4
Unique Categories: ['TA' 'Gd' 'Fa' 'Po']
Missing Values: 0

One-Hot Encoded DataFrame:
      BsmtCond_Gd  BsmtCond_Po  BsmtCond_TA
0           False        False         True
1           False        False         True
2           False        False         True
3           False        False         True
4           False        False         True
...           ...          ...          ...
1016        False        False         True
1017        False        False         True
1018        False        False         True
1020        False        False         True
1021        False        False         True

[993 rows x 3 columns]
BsmtCond_Gd Correlation with SalePrice: 0.07166047403638387
BsmtCond_Po Correlation with SalePrice: -0.06617021667973803
BsmtCond_TA Correlation with SalePrice: 0.0443161917096095


In [50]:
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'BsmtCond')
droppedCols.append('BsmtCond')

Number of continuous features: 36
Number of categorical features: 20
Total Features: 61


# 24

In [51]:
inspectCol(categorical_columns[0], DF)
percentMissing(categorical_columns[0], DF)
DF = DF.dropna(subset=['BsmtExposure'])
percentMissing(categorical_columns[0], DF)
DF_with_processed_categorical = process_categorical_column(DF, categorical_columns[0], target_variable='SalePrice')

Inspecting Column: BsmtExposure
Data Type: object
Number of Unique Categories: 5
Unique Categories: ['No' 'Mn' 'Gd' 'Av' nan]
Missing Values: 1

Missing rate of BsmtExposure: 0.0010070493454179255
Missing rate of BsmtExposure: 0.0
One-Hot Encoded DataFrame:
      BsmtExposure_Gd  BsmtExposure_Mn  BsmtExposure_No
0               False            False             True
1               False             True            False
2                True            False            False
3               False            False             True
4               False            False             True
...               ...              ...              ...
1016            False            False             True
1017            False            False             True
1018            False            False             True
1020            False            False             True
1021            False            False             True

[992 rows x 3 columns]
BsmtExposure_Gd Correlation with SalePrice: 0.

In [52]:
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'BsmtExposure')
droppedCols.append('BsmtExposure')

Number of continuous features: 36
Number of categorical features: 19
Total Features: 60


# 25

In [53]:
inspectCol(categorical_columns[0], DF)
DF_with_processed_categorical = process_categorical_column(DF, categorical_columns[0], target_variable='SalePrice')

Inspecting Column: BsmtFinType1
Data Type: object
Number of Unique Categories: 6
Unique Categories: ['GLQ' 'Unf' 'Rec' 'LwQ' 'BLQ' 'ALQ']
Missing Values: 0

One-Hot Encoded DataFrame:
      BsmtFinType1_BLQ  BsmtFinType1_GLQ  BsmtFinType1_LwQ  BsmtFinType1_Rec  \
0                False              True             False             False   
1                False             False             False             False   
2                False             False             False              True   
3                False              True             False             False   
4                False              True             False             False   
...                ...               ...               ...               ...   
1016             False             False             False              True   
1017             False             False             False             False   
1018             False             False             False             False   
1020            

In [54]:
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'BsmtFinType1')
droppedCols.append('BsmtFinType1')

Number of continuous features: 36
Number of categorical features: 18
Total Features: 59


# 26

In [55]:
inspectCol(categorical_columns[0], DF)
percentMissing(categorical_columns[0], DF)
DF = DF.dropna(subset=['BsmtFinType2'])
percentMissing(categorical_columns[0], DF)
DF_with_processed_categorical = process_categorical_column(DF, categorical_columns[0], target_variable='SalePrice')

Inspecting Column: BsmtFinType2
Data Type: object
Number of Unique Categories: 7
Unique Categories: ['Unf' 'ALQ' 'BLQ' 'GLQ' 'Rec' 'LwQ' nan]
Missing Values: 1

Missing rate of BsmtFinType2: 0.0010080645161290322
Missing rate of BsmtFinType2: 0.0
One-Hot Encoded DataFrame:
      BsmtFinType2_BLQ  BsmtFinType2_GLQ  BsmtFinType2_LwQ  BsmtFinType2_Rec  \
0                False             False             False             False   
1                False             False             False             False   
2                False             False             False             False   
3                False             False             False             False   
4                False             False             False             False   
...                ...               ...               ...               ...   
1016             False             False             False             False   
1017             False             False             False             False   
1018  

In [56]:
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'BsmtFinType2')
droppedCols.append('BsmtFinType2')

Number of continuous features: 36
Number of categorical features: 17
Total Features: 58


# 27

In [57]:
inspectCol(categorical_columns[0], DF)
DF_with_processed_categorical = process_categorical_column(DF, categorical_columns[0], target_variable='SalePrice')

Inspecting Column: Heating
Data Type: object
Number of Unique Categories: 4
Unique Categories: ['GasA' 'Grav' 'GasW' 'OthW']
Missing Values: 0

One-Hot Encoded DataFrame:
      Heating_GasW  Heating_Grav  Heating_OthW
0            False         False         False
1            False         False         False
2            False         False         False
3            False         False         False
4            False         False         False
...            ...           ...           ...
1016         False         False         False
1017         False         False         False
1018         False         False         False
1020         False         False         False
1021         False         False         False

[991 rows x 3 columns]
Heating_GasW Correlation with SalePrice: -0.01506347367794061
Heating_Grav Correlation with SalePrice: -0.10663215994905462
Heating_OthW Correlation with SalePrice: -0.0318813186480591


In [58]:
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'Heating')
droppedCols.append('Heating')

Number of continuous features: 36
Number of categorical features: 16
Total Features: 57


# 28

In [59]:
inspectCol(categorical_columns[0], DF)
DF_with_processed_categorical = process_categorical_column(DF, categorical_columns[0], target_variable='SalePrice')

Inspecting Column: HeatingQC
Data Type: object
Number of Unique Categories: 5
Unique Categories: ['Ex' 'TA' 'Gd' 'Fa' 'Po']
Missing Values: 0

One-Hot Encoded DataFrame:
      HeatingQC_Fa  HeatingQC_Gd  HeatingQC_Po  HeatingQC_TA
0            False         False         False         False
1            False         False         False          True
2            False         False         False          True
3            False         False         False         False
4            False         False         False          True
...            ...           ...           ...           ...
1016         False         False         False         False
1017         False         False         False          True
1018         False          True         False         False
1020         False          True         False         False
1021         False          True         False         False

[991 rows x 4 columns]
HeatingQC_Fa Correlation with SalePrice: -0.12311185420022484
HeatingQC_Gd

In [60]:
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'HeatingQC')
droppedCols.append('HeatingQC')

Number of continuous features: 36
Number of categorical features: 15
Total Features: 56


In [61]:
# 29
inspectCol(categorical_columns[0], DF)
DF_with_processed_categorical = process_categorical_column(DF, categorical_columns[0], target_variable='SalePrice')

Inspecting Column: CentralAir
Data Type: object
Number of Unique Categories: 2
Unique Categories: ['Y' 'N']
Missing Values: 0

One-Hot Encoded DataFrame:
      CentralAir_Y
0             True
1             True
2             True
3             True
4             True
...            ...
1016          True
1017          True
1018          True
1020          True
1021          True

[991 rows x 1 columns]
CentralAir_Y Correlation with SalePrice: 0.24614751504296462


In [62]:
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'CentralAir')
droppedCols.append('CentralAir')

Number of continuous features: 36
Number of categorical features: 14
Total Features: 55


# 30

In [63]:
inspectCol(categorical_columns[0], DF)
DF_with_processed_categorical = process_categorical_column(DF, categorical_columns[0], target_variable='SalePrice')

Inspecting Column: Electrical
Data Type: object
Number of Unique Categories: 5
Unique Categories: ['SBrkr' 'FuseA' 'Mix' 'FuseF' 'FuseP']
Missing Values: 0

One-Hot Encoded DataFrame:
      Electrical_FuseF  Electrical_FuseP  Electrical_Mix  Electrical_SBrkr
0                False             False           False              True
1                False             False           False              True
2                False             False           False              True
3                False             False           False              True
4                False             False           False             False
...                ...               ...             ...               ...
1016             False             False           False              True
1017             False             False           False              True
1018             False             False           False              True
1020             False             False           False          

In [64]:
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'Electrical')
droppedCols.append('Electrical')

Number of continuous features: 36
Number of categorical features: 13
Total Features: 54


# 31

In [65]:
inspectCol(categorical_columns[0], DF)
DF_with_processed_categorical = process_categorical_column(DF, categorical_columns[0], target_variable='SalePrice')

Inspecting Column: KitchenQual
Data Type: object
Number of Unique Categories: 4
Unique Categories: ['Gd' 'TA' 'Fa' 'Ex']
Missing Values: 0

One-Hot Encoded DataFrame:
      KitchenQual_Fa  KitchenQual_Gd  KitchenQual_TA
0              False            True           False
1              False           False            True
2              False           False            True
3              False            True           False
4              False           False            True
...              ...             ...             ...
1016           False           False            True
1017           False            True           False
1018           False           False            True
1020           False           False            True
1021           False           False            True

[991 rows x 3 columns]
KitchenQual_Fa Correlation with SalePrice: -0.15789326890842414
KitchenQual_Gd Correlation with SalePrice: 0.2822362503032197
KitchenQual_TA Correlation with SalePrice: -0.5

In [66]:
DF['KitchenQual_TA'] = DF_with_processed_categorical['KitchenQual_TA']
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'KitchenQual')
droppedCols.append('KitchenQual')


Number of continuous features: 36
Number of categorical features: 12
Total Features: 54


# 32

In [67]:
inspectCol(categorical_columns[0], DF)
DF_with_processed_categorical = process_categorical_column(DF, categorical_columns[0], target_variable='SalePrice')

Inspecting Column: Functional
Data Type: object
Number of Unique Categories: 6
Unique Categories: ['Typ' 'Min1' 'Mod' 'Maj1' 'Maj2' 'Min2']
Missing Values: 0

One-Hot Encoded DataFrame:
      Functional_Maj2  Functional_Min1  Functional_Min2  Functional_Mod  \
0               False            False            False           False   
1               False            False            False           False   
2               False            False            False           False   
3               False            False            False           False   
4               False            False            False           False   
...               ...              ...              ...             ...   
1016            False            False            False           False   
1017            False            False            False           False   
1018            False            False            False           False   
1020            False            False             True         

In [68]:
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'Functional')
droppedCols.append('Functional')

Number of continuous features: 36
Number of categorical features: 11
Total Features: 53


# 33

In [69]:
inspectCol(categorical_columns[0], DF)
percentMissing(categorical_columns[0], DF)
DF = DF.dropna(subset=['FireplaceQu'])
percentMissing(categorical_columns[0], DF)
DF_with_processed_categorical = process_categorical_column(DF, categorical_columns[0], target_variable='SalePrice')

Inspecting Column: FireplaceQu
Data Type: object
Number of Unique Categories: 6
Unique Categories: [nan 'Gd' 'TA' 'Po' 'Fa' 'Ex']
Missing Values: 459

Missing rate of FireplaceQu: 0.46316851664984865
Missing rate of FireplaceQu: 0.0
One-Hot Encoded DataFrame:
      FireplaceQu_Fa  FireplaceQu_Gd  FireplaceQu_Po  FireplaceQu_TA
1              False            True           False           False
2              False           False           False            True
3              False            True           False           False
5              False           False           False            True
7              False           False           False            True
...              ...             ...             ...             ...
1015           False           False           False            True
1017           False           False           False            True
1018           False            True           False           False
1020           False           False           Fal

In [70]:
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'FireplaceQu')
droppedCols.append('FireplaceQu')

Number of continuous features: 36
Number of categorical features: 10
Total Features: 52


# 34

In [71]:
inspectCol(categorical_columns[0], DF)
percentMissing(categorical_columns[0], DF)
DF = DF.dropna(subset=['GarageType'])
percentMissing(categorical_columns[0], DF)
DF_with_processed_categorical = process_categorical_column(DF, categorical_columns[0], target_variable='SalePrice')

Inspecting Column: GarageType
Data Type: object
Number of Unique Categories: 6
Unique Categories: ['Detchd' 'Attchd' 'BuiltIn' nan 'Basment' '2Types']
Missing Values: 6

Missing rate of GarageType: 0.011278195488721804
Missing rate of GarageType: 0.0
One-Hot Encoded DataFrame:
      GarageType_Attchd  GarageType_Basment  GarageType_BuiltIn  \
1                 False               False               False   
2                  True               False               False   
3                  True               False               False   
5                  True               False               False   
7                  True               False               False   
...                 ...                 ...                 ...   
1015               True               False               False   
1017              False               False               False   
1018               True               False               False   
1020              False               False         

In [72]:
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'GarageType')
droppedCols.append('GarageType')

Number of continuous features: 36
Number of categorical features: 9
Total Features: 51


In [73]:
#35
inspectCol(categorical_columns[0], DF)
DF_with_processed_categorical = process_categorical_column(DF, categorical_columns[0], target_variable='SalePrice')

Inspecting Column: GarageFinish
Data Type: object
Number of Unique Categories: 3
Unique Categories: ['Unf' 'Fin' 'RFn']
Missing Values: 0

One-Hot Encoded DataFrame:
      GarageFinish_RFn  GarageFinish_Unf
1                False              True
2                False              True
3                False             False
5                 True             False
7                 True             False
...                ...               ...
1015              True             False
1017             False              True
1018             False              True
1020             False              True
1021             False              True

[526 rows x 2 columns]
GarageFinish_RFn Correlation with SalePrice: -0.027658255182225276
GarageFinish_Unf Correlation with SalePrice: -0.35384694222854846


In [74]:
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'GarageFinish')
droppedCols.append('GarageFinish')

Number of continuous features: 36
Number of categorical features: 8
Total Features: 50


In [75]:
#36
inspectCol(categorical_columns[0], DF)
DF_with_processed_categorical = process_categorical_column(DF, categorical_columns[0], target_variable='SalePrice')

Inspecting Column: GarageQual
Data Type: object
Number of Unique Categories: 4
Unique Categories: ['Fa' 'TA' 'Gd' 'Ex']
Missing Values: 0

One-Hot Encoded DataFrame:
      GarageQual_Fa  GarageQual_Gd  GarageQual_TA
1              True          False          False
2             False          False           True
3             False          False           True
5             False          False           True
7             False          False           True
...             ...            ...            ...
1015          False          False           True
1017          False          False           True
1018          False          False           True
1020          False          False           True
1021          False          False           True

[526 rows x 3 columns]
GarageQual_Fa Correlation with SalePrice: -0.10680580910563431
GarageQual_Gd Correlation with SalePrice: 0.008640090523538067
GarageQual_TA Correlation with SalePrice: 0.08398154269817613


In [76]:
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'GarageQual')
droppedCols.append('GarageQual')

Number of continuous features: 36
Number of categorical features: 7
Total Features: 49


In [77]:
#37
inspectCol(categorical_columns[0], DF)
DF_with_processed_categorical = process_categorical_column(DF, categorical_columns[0], target_variable='SalePrice')

Inspecting Column: GarageCond
Data Type: object
Number of Unique Categories: 5
Unique Categories: ['Po' 'TA' 'Gd' 'Fa' 'Ex']
Missing Values: 0

One-Hot Encoded DataFrame:
      GarageCond_Fa  GarageCond_Gd  GarageCond_Po  GarageCond_TA
1             False          False           True          False
2             False          False          False           True
3             False          False          False           True
5             False          False          False           True
7             False          False          False           True
...             ...            ...            ...            ...
1015          False          False          False           True
1017          False          False          False           True
1018          False          False          False           True
1020          False          False          False           True
1021          False          False          False           True

[526 rows x 4 columns]
GarageCond_Fa Correlation

In [78]:
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'GarageCond')
droppedCols.append('GarageCond')

Number of continuous features: 36
Number of categorical features: 6
Total Features: 48


In [79]:
#38
inspectCol(categorical_columns[0], DF)
DF_with_processed_categorical = process_categorical_column(DF, categorical_columns[0], target_variable='SalePrice')

Inspecting Column: PavedDrive
Data Type: object
Number of Unique Categories: 3
Unique Categories: ['P' 'Y' 'N']
Missing Values: 0

One-Hot Encoded DataFrame:
      PavedDrive_P  PavedDrive_Y
1             True         False
2            False          True
3            False          True
5            False          True
7            False          True
...            ...           ...
1015         False          True
1017         False          True
1018         False          True
1020         False          True
1021         False          True

[526 rows x 2 columns]
PavedDrive_P Correlation with SalePrice: -0.1350192528388644
PavedDrive_Y Correlation with SalePrice: 0.14423600725414423


In [80]:
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'PavedDrive')
droppedCols.append('PavedDrive')

Number of continuous features: 36
Number of categorical features: 5
Total Features: 47


In [81]:
#39
inspectCol(categorical_columns[0], DF)
percentMissing(categorical_columns[0], DF)
DF_with_processed_categorical = process_categorical_column(DF, categorical_columns[0], target_variable='SalePrice')

Inspecting Column: PoolQC
Data Type: object
Number of Unique Categories: 4
Unique Categories: [nan 'Gd' 'Ex' 'Fa']
Missing Values: 521

Missing rate of PoolQC: 0.9904942965779467
One-Hot Encoded DataFrame:
      PoolQC_Fa  PoolQC_Gd
1         False      False
2         False      False
3         False      False
5         False      False
7         False      False
...         ...        ...
1015      False      False
1017      False      False
1018      False      False
1020      False      False
1021      False      False

[526 rows x 2 columns]
PoolQC_Fa Correlation with SalePrice: -0.018919149516509297
PoolQC_Gd Correlation with SalePrice: -0.001332181600408204


In [82]:
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'PoolQC')
droppedCols.append('PoolQC')

Number of continuous features: 36
Number of categorical features: 4
Total Features: 46


In [83]:
inspectCol(categorical_columns[0], DF)
percentMissing(categorical_columns[0], DF)
DF_with_processed_categorical = process_categorical_column(DF, categorical_columns[0], target_variable='SalePrice')

Inspecting Column: Fence
Data Type: object
Number of Unique Categories: 5
Unique Categories: ['MnPrv' 'GdWo' nan 'GdPrv' 'MnWw']
Missing Values: 440

Missing rate of Fence: 0.8365019011406845
One-Hot Encoded DataFrame:
      Fence_GdWo  Fence_MnPrv  Fence_MnWw
1          False         True       False
2           True        False       False
3          False        False       False
5          False        False       False
7          False        False       False
...          ...          ...         ...
1015       False        False       False
1017       False        False       False
1018       False         True       False
1020       False        False       False
1021        True        False       False

[526 rows x 3 columns]
Fence_GdWo Correlation with SalePrice: -0.10856836634691885
Fence_MnPrv Correlation with SalePrice: -0.1547196387780182
Fence_MnWw Correlation with SalePrice: -0.0449167823911656


In [84]:
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'Fence')
droppedCols.append('Fence')

Number of continuous features: 36
Number of categorical features: 3
Total Features: 45


In [85]:
inspectCol(categorical_columns[0], DF)
percentMissing(categorical_columns[0], DF)
DF_with_processed_categorical = process_categorical_column(DF, categorical_columns[0], target_variable='SalePrice')

Inspecting Column: MiscFeature
Data Type: object
Number of Unique Categories: 4
Unique Categories: [nan 'Shed' 'Gar2' 'Othr']
Missing Values: 511

Missing rate of MiscFeature: 0.9714828897338403
One-Hot Encoded DataFrame:
      MiscFeature_Othr  MiscFeature_Shed
1                False             False
2                False             False
3                False             False
5                False             False
7                False             False
...                ...               ...
1015             False             False
1017             False             False
1018             False             False
1020             False             False
1021             False             False

[526 rows x 2 columns]
MiscFeature_Othr Correlation with SalePrice: -0.04257123988367608
MiscFeature_Shed Correlation with SalePrice: -0.08072419921386656


In [86]:
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'MiscFeature')
droppedCols.append('MiscFeature')

Number of continuous features: 36
Number of categorical features: 2
Total Features: 44


In [87]:
inspectCol(categorical_columns[0], DF)
percentMissing(categorical_columns[0], DF)
DF_with_processed_categorical = process_categorical_column(DF, categorical_columns[0], target_variable='SalePrice')

Inspecting Column: SaleType
Data Type: object
Number of Unique Categories: 7
Unique Categories: ['WD' 'New' 'COD' 'CWD' 'ConLw' 'ConLD' 'Con']
Missing Values: 0

Missing rate of SaleType: 0.0
One-Hot Encoded DataFrame:
      SaleType_CWD  SaleType_Con  SaleType_ConLD  SaleType_ConLw  \
1            False         False           False           False   
2            False         False           False           False   
3            False         False           False           False   
5            False         False           False           False   
7            False         False           False           False   
...            ...           ...             ...             ...   
1015         False         False           False           False   
1017         False         False           False           False   
1018         False         False           False           False   
1020         False         False           False           False   
1021         False         False 

In [88]:
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'SaleType')
droppedCols.append('SaleType')

Number of continuous features: 36
Number of categorical features: 1
Total Features: 43


In [89]:
inspectCol(categorical_columns[0], DF)

DF_with_processed_categorical = process_categorical_column(DF, categorical_columns[0], target_variable='SalePrice')

Inspecting Column: SaleCondition
Data Type: object
Number of Unique Categories: 5
Unique Categories: ['Normal' 'Partial' 'Abnorml' 'Family' 'Alloca']
Missing Values: 0

One-Hot Encoded DataFrame:
      SaleCondition_Alloca  SaleCondition_Family  SaleCondition_Normal  \
1                    False                 False                  True   
2                    False                 False                  True   
3                    False                 False                  True   
5                    False                 False                  True   
7                    False                 False                  True   
...                    ...                   ...                   ...   
1015                 False                 False                  True   
1017                 False                 False                  True   
1018                 False                 False                  True   
1020                 False                 False                

In [90]:
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'SaleCondition')
droppedCols.append('SaleCondition')

Number of continuous features: 36
Number of categorical features: 0
Total Features: 42


In [91]:
print("Total Dropped Categorical Variables", len(droppedCols))


Total Dropped Categorical Variables 47


# Continous Variables
-Inspect and check correlation and see how many we have left. 

In [92]:
droppedCont = []
def inspectContinous(col, df):
    print("Name:", col)
    print(df[col])
    print("Missing values", df[col].isnull().sum())
    print("Correlation to target", check_correlation(df[[col, 'SalePrice']], target_variable='SalePrice') )
    print("Unique Values", df[col].unique())

In [93]:
inspectContinous(numerical_columns[0], DF)
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'Unnamed: 0')
droppedCont.append('Unnamed: 0')

Name: Unnamed: 0
1        651
2        260
3       1089
5        881
7        672
        ... 
1015     832
1017     962
1018     135
1020     668
1021     462
Name: Unnamed: 0, Length: 526, dtype: int64
Missing values 0
Correlation to target             SalePrice
SalePrice    1.000000
Unnamed: 0  -0.063743
Unique Values [ 651  260 1089  881  672 1402  841  203  844  166 1424 1389  239 1361
  719  939    4   70  522  262   95  414  616 1230  524  816 1209  467
  228 1243  851  890 1271 1224  369  538 1075  941  417  442   55 1300
  944  783  791  272  626 1312  700  364  400 1203   25   33 1012  530
  564 1147 1318  120 1063 1327   24  440 1148  336  639    1  310  174
 1024 1059  167 1433   14  732  359  196  854  413  765   34  118    6
  343  294  114  357 1138  157  271  534  763 1367 1082 1393  857  315
 1315   93  802  708 1329  222 1284 1344 1255 1043  267 1375  795  981
  268  891  803  623 1162 1394  848  689  780 1120  273 1145    9  682
  351  583 1128 1273 1250  152  661  7

In [94]:
inspectContinous(numerical_columns[0], DF)
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'Id')
droppedCont.append('Id')

Name: Id
1        652
2        261
3       1090
5        882
7        673
        ... 
1015     833
1017     963
1018     136
1020     669
1021     463
Name: Id, Length: 526, dtype: int64
Missing values 0
Correlation to target            SalePrice
SalePrice   1.000000
Id         -0.063743
Unique Values [ 652  261 1090  882  673 1403  842  204  845  167 1425 1390  240 1362
  720  940    5   71  523  263   96  415  617 1231  525  817 1210  468
  229 1244  852  891 1272 1225  370  539 1076  942  418  443   56 1301
  945  784  792  273  627 1313  701  365  401 1204   26   34 1013  531
  565 1148 1319  121 1064 1328   25  441 1149  337  640    2  311  175
 1025 1060  168 1434   15  733  360  197  855  414  766   35  119    7
  344  295  115  358 1139  158  272  535  764 1368 1083 1394  858  316
 1316   94  803  709 1330  223 1285 1345 1256 1044  268 1376  796  982
  269  892  804  624 1163 1395  849  690  781 1121  274 1146   10  683
  352  584 1129 1274 1251  153  662  708 1314   29  598  

In [95]:
inspectContinous(numerical_columns[0], DF)
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'MSSubClass')
droppedCont.append('MSSubClass')

Name: MSSubClass
1        70
2        80
3       120
5        50
7        20
       ... 
1015     60
1017    160
1018     20
1020     20
1021     20
Name: MSSubClass, Length: 526, dtype: int64
Missing values 0
Correlation to target             SalePrice
SalePrice    1.000000
MSSubClass  -0.082495
Unique Values [ 70  80 120  50  20  60  90  85  30 160 190  75  40  45]
Number of continuous features: 33
Number of categorical features: 0
Total Features: 39


In [96]:
inspectContinous(numerical_columns[0], DF)
#normalize


Name: LotArea
1        9084
2       19296
3        3316
5       13758
7       11250
        ...  
1015     9548
1017     2308
1018    10400
1020    14175
1021     8281
Name: LotArea, Length: 526, dtype: int64
Missing values 0
Correlation to target            SalePrice
SalePrice   1.000000
LotArea     0.197499
Unique Values [  9084  19296   3316  13758  11250   6762  10440   4438  12665  10708
   9503   6000   8741  16158   9920  24090  14260  13651   5000   8471
   9765  11228   7861  18890  11787  11425  10182   9480   8521  13891
   3196   8064   9156  15384   9830  11553  13125   8755  22420   6240
  10175  10762  14375   9101  11333  11764  12342   9572  14331  18800
  14963   9750  14230  10552  10592  10200  13346  12000  14781  21453
   6600   8246  15431   5700  14157   3982   9600   7685  12416  15498
  11275  10562  10261  10920  11404  12011   9416  17920   8960  14587
   7313  12376  10084   8849   7259   4224   9819  12003  39104   9056
   9430   2665   8749  10800   8125 

In [97]:
inspectContinous(numerical_columns[1], DF)

Name: OverallQual
1       4
2       6
3       8
5       7
7       6
       ..
1015    7
1017    6
1018    7
1020    5
1021    5
Name: OverallQual, Length: 526, dtype: int64
Missing values 0
Correlation to target              SalePrice
SalePrice     1.000000
OverallQual   0.757005
Unique Values [ 4  6  8  7  5 10  9  3]


In [98]:
inspectContinous(numerical_columns[2], DF)

Name: OverallCond
1       5
2       5
3       5
5       5
7       6
       ..
1015    6
1017    6
1018    6
1020    6
1021    5
Name: OverallCond, Length: 526, dtype: int64
Missing values 0
Correlation to target              SalePrice
SalePrice     1.000000
OverallCond  -0.112766
Unique Values [5 6 8 4 7 9 2 3]


In [99]:
inspectContinous(numerical_columns[3], DF)

Name: YearBuilt
1       1940
2       1962
3       2005
5       1990
7       1977
        ... 
1015    2003
1017    1976
1018    1970
1020    1956
1021    1965
Name: YearBuilt, Length: 526, dtype: int64
Missing values 0
Correlation to target            SalePrice
SalePrice   1.000000
YearBuilt   0.475155
Unique Values [1940 1962 2005 1990 1977 2006 1904 2004 1915 1955 1958 1941 1945 1969
 2000 1973 1947 1993 2002 1996 1954 1942 1967 2003 1949 1968 1959 1999
 1918 1930 1964 1978 1976 1960 2007 1923 1988 1992 2001 1925 1982 2008
 1926 1986 1932 1998 1927 1953 2009 1905 1994 1910 1975 1919 1931 1939
 1980 1908 1995 1920 1928 1893 1971 1957 1922 1997 1965 1979 1916 1983
 1950 1936 1984 1934 1970 1972 1974 1935 1963 1966 1985 1929 1948 1956
 1961 2010 1921 1989 1938 1987 1991 1914 1937 1981 1913 1917]


In [100]:
inspectContinous(numerical_columns[4], DF)

Name: YearRemodAdd
1       1950
2       1962
3       2005
5       1991
7       1977
        ... 
1015    2003
1017    1976
1018    1970
1020    1987
1021    1965
Name: YearRemodAdd, Length: 526, dtype: int64
Missing values 0
Correlation to target               SalePrice
SalePrice      1.000000
YearRemodAdd   0.499669
Unique Values [1950 1962 2005 1991 1977 2006 2002 2004 1993 1983 1969 2000 1973 1995
 2003 1997 1954 1967 1968 1984 1999 1992 1964 1958 1978 1976 2007 1990
 1996 2001 1959 1989 1980 2008 1994 1986 1960 1998 1974 1953 2010 1975
 1981 1955 2009 1988 1971 1965 1979 1972 1963 1985 1987 1956 1970 1961
 1966 1957]


In [101]:
inspectContinous(numerical_columns[5], DF)

Name: BsmtFinSF1
1          0
2        672
3       1039
5        902
7        767
        ... 
1015     483
1017     556
1018       0
1020     988
1021     553
Name: BsmtFinSF1, Length: 526, dtype: int64
Missing values 0
Correlation to target             SalePrice
SalePrice    1.000000
BsmtFinSF1   0.359626
Unique Values [   0  672 1039  902  767  662  379  457  375   94 1274  523  655 1880
  399  506  310   50  498  719  486 1220  386  842 1386  724   72  673
  410  772 1128  490  694  111 1097  539  524  482  712  786 1018  783
  728  275  938  397  641  188 1767 1249 1154  978  518 1398 1165  297
 1288  733  252  956 1126  306 1153 1470 1369   28 1285  774  874 1567
  226 1163  548  425  500 1046  648  550 1085  378  854  403  420  492
 1070   20  620  851 1003 1082 1065  416 1361  863 1277  831   63   16
 1218  649  970  580  775  547 1904 1258  364   24  690 1441  485 1309
 1170 1416  975  168  371  626 1513  584  828  912 1110  705  419  998
  593 1455   57  639  885  988 1410  3

In [102]:
inspectContinous(numerical_columns[6], DF)
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'BsmtFinSF2')
droppedCont.append('BsmtFinSF2')

Name: BsmtFinSF2
1         0
2       690
3         0
5         0
7         0
       ... 
1015      0
1017      0
1018      0
1020      0
1021    311
Name: BsmtFinSF2, Length: 526, dtype: int64
Missing values 0
Correlation to target             SalePrice
SalePrice    1.000000
BsmtFinSF2  -0.019006
Unique Values [   0  690  768  374  531  211  258  354  668  539  208  400  557 1085
  150 1063  173  276  125  500  620  180  147  165  177  546  182  441
  168  110  163   81  713 1057  344  117 1127  219  551  127  823  972
  184  377  682  215  634  820  547   32  230  228  391   68   41  841
  661  193 1031  132  311]
Number of continuous features: 32
Number of categorical features: 0
Total Features: 38


In [103]:
inspectContinous(numerical_columns[6], DF)
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'BsmtUnfSF')
droppedCont.append('BsmtUnfSF')

Name: BsmtUnfSF
1        755
2          0
3        208
5        254
7        441
        ... 
1015     458
1017     248
1018    1304
1020     200
1021       0
Name: BsmtUnfSF, Length: 526, dtype: int64
Missing values 0
Correlation to target            SalePrice
SalePrice    1.00000
BsmtUnfSF    0.16149
Unique Values [ 755    0  208  254  441 1286  650  186  876  470  193  360  641  256
  448 1032  490  343  605  370  499  326  652  660  522  440  342   70
  690 1273  672 1489   64  733  378  390  220  242 1078  935  284  354
  628  978  971  526   84  474 1630 1566  380  602  678  367  429 1753
  706  175  204  788  572  673  366  179  294  936  520  901  130  600
  372 1008 1498  408  203  317 1656  131  104  268  774   96  707   89
   36 1459  482  702  291  176  168   80  728  163  804  459 1571  371
  212  192 1734  336  402  286  907 1342 1237 1012 1042  140  288  847
  300 1045  409   88  461 1393  207 1258  457  320 1560  588  712  350
 1141 1357 1352 1926 1580  166  103  333  2

In [104]:
inspectContinous(numerical_columns[6], DF)

Name: TotalBsmtSF
1        755
2       1362
3       1247
5       1156
7       1208
        ... 
1015     941
1017     804
1018    1304
1020    1188
1021     864
Name: TotalBsmtSF, Length: 526, dtype: int64
Missing values 0
Correlation to target              SalePrice
SalePrice     1.000000
TotalBsmtSF   0.558563
Unique Values [ 755 1362 1247 1156 1208 1286  650  848  876 1617 1024  735 1530  971
 1032 1145 2223 1004  506  680 1080  783 1361 1379 1008 1660  728  912
 2076 1273  672 1489  788 1063 1051  800  992 1370 1078 1425  978  819
 1097 1029 1152 1453 1800  796 1260 1630 1566 1398  602 1461 1095  704
 1753  938 1103  816 1060 3094  572 1922 1520 1262  697 1606 1565  854
 1582  936 1253 1153 1086 1726 1763 1498 1561 1673 1686 1684 1416 1028
 1142 1567  774 1385  707 1252  757 1459  482  702  716  952 1214  935
  713  804  884 1475  720 1571 1225  740  918 1734  756  894 1356  907
 1237 1012 1240 1042  991 1291  847 1019 2110  825 1629 1324 1393 1484
 1258  832 1151 1623  588 1568 11

In [105]:
inspectContinous(numerical_columns[7], DF)

Name: 1stFlrSF
1        755
2       1382
3       1247
5       1187
7       1208
        ... 
1015     941
1017     804
1018    1682
1020    1437
1021     864
Name: 1stFlrSF, Length: 526, dtype: int64
Missing values 0
Correlation to target            SalePrice
SalePrice   1.000000
1stFlrSF    0.552353
Unique Values [ 755 1382 1247 1187 1208 1294  958  848  876 1867 1344  869  798 1530
  971 1207 1145 2223 1004 1212  680 1080  807 1361 1383 1008 1660  888
  912 2076 1456  672 1489  788 1287 1159  960 1022 1370 1128 1425 1005
 1110 1062 1164 1422 1453 1800  790 1288 1630 1600 1700  900 1509 1166
  860 1787  988 1103  816 1060 2402  572 1922 1567 1262  697 1651 2898
 1096 1610  962 1253 1153 1086 1726 1779 1028 1506 1561 1699 1694 1684
 1644 1436 1142  774 1363  707 1268  925 1459 1221  702  716 1092 1260
  728  955  811  959 1532 1052 1571 1276  778  918 1734  756  894 1500
 1307 1362 1012 1632 1042 1077 1291 1521  847 1357 2110 1097 1686 1324
 1402 1036 1636 1423  832 1689 1165 1555 1352

In [106]:
inspectContinous(numerical_columns[8], DF)
DF, categorical_columns, numerical_columns = updateDataFrame(DF, '2ndFlrSF')
droppedCont.append('2ndFlrSF')

Name: 2ndFlrSF
1       755
2         0
3         0
5       530
7         0
       ... 
1015    888
1017    744
1018      0
1020      0
1021      0
Name: 2ndFlrSF, Length: 526, dtype: int64
Missing values 0
Correlation to target            SalePrice
SalePrice   1.000000
2ndFlrSF    0.255562
Unique Values [ 755    0  530  581  540  349  689 1196 1053  660  790 1017  702 1259
 1015  756  252  780 1038 1254  445  978 1106 1357  784  602 1129  704
  539  804  895  551  830  878  838 1523  884 1194  707 1097  550  691
  779  716 1020 1031  728  677  741  748  408  797  720  676 1336  765
 1088 1051  534 1112  896  762 1177  809 1098  975 1174  886 1332  167
  510 1178  192  875  620 1081  651  636  504 1589  989  766  862  370
  844  900  587 1519  482  729 1092  882 1872  668 1330  514  464  880
  685  876  910  850 1070 1134 1306 1140  511 1054  846  590 1479  832
  874  983 1218 1080  936  698 1286  795  725  742  670  834  831  574
  546  930  694  769  857 1237  988  752 1274  866  208 

In [107]:
inspectContinous(numerical_columns[8], DF)
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'LowQualFinSF')
droppedCont.append('LowQualFinSF')

Name: LowQualFinSF
1       0
2       0
3       0
5       0
7       0
       ..
1015    0
1017    0
1018    0
1020    0
1021    0
Name: LowQualFinSF, Length: 526, dtype: int64
Missing values 0
Correlation to target               SalePrice
SalePrice       1.00000
LowQualFinSF   -0.04827
Unique Values [  0 420 232 384 360 392]
Number of continuous features: 29
Number of categorical features: 0
Total Features: 35


In [108]:
inspectContinous(numerical_columns[8], DF)


Name: GrLivArea
1       1510
2       1382
3       1247
5       1717
7       1208
        ... 
1015    1829
1017    1548
1018    1682
1020    1437
1021     864
Name: GrLivArea, Length: 526, dtype: int64
Missing values 0
Correlation to target            SalePrice
SalePrice   1.000000
GrLivArea   0.677742
Unique Values [1510 1382 1247 1717 1208 1294 1539  848 1416 1867 1344 1218 1487 1530
  971 2403 2198 2223 1664 1212 1470 2097 1509 2620 2398 1008 1660 1644
  912 2076 1456  924 1489 1490 1287 1159 1740 2060 2624 1573 1425 1983
 1110 1062 2270 1422 2810 1800 1574 1288 1630 1600 1700 1502 2295 1564
 1787  988 1103  816 1060 2402 1111 1922 1567 1262 1501 1651 2898 1991
 2161 1792 1253 2031 1924 1726 1779 1028 1506 1561 3222 1694 1684 2320
 1142 1968 1363 1414 2365 1475 1459 1912 1481 1432 2112 2291 1632 1552
 2447 1367 2329 2192 1571 2612  778 1683 2822 1512  894 1500 2358 1362
 1012 1576 1077 1291 1453 2775 1959 1357 2110 1993 2448 1324 2599 1402
 1268 1636 2555 1641 1689 2263 1555 1352 19

In [109]:
inspectContinous(numerical_columns[9], DF)
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'BsmtFullBath')
droppedCont.append('BsmtFullBath')

Name: BsmtFullBath
1       1
2       1
3       1
5       0
7       1
       ..
1015    1
1017    1
1018    0
1020    1
1021    0
Name: BsmtFullBath, Length: 526, dtype: int64
Missing values 0
Correlation to target               SalePrice
SalePrice      1.000000
BsmtFullBath   0.209774
Unique Values [1 0 2]
Number of continuous features: 28
Number of categorical features: 0
Total Features: 34


In [110]:
inspectContinous(numerical_columns[9], DF)
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'BsmtHalfBath')
droppedCont.append('BsmtHalfBath')

Name: BsmtHalfBath
1       0
2       0
3       0
5       0
7       0
       ..
1015    0
1017    0
1018    0
1020    0
1021    0
Name: BsmtHalfBath, Length: 526, dtype: int64
Missing values 0
Correlation to target               SalePrice
SalePrice       1.00000
BsmtHalfBath   -0.05027
Unique Values [0 1 2]
Number of continuous features: 27
Number of categorical features: 0
Total Features: 33


In [111]:
inspectContinous(numerical_columns[9], DF)

Name: FullBath
1       1
2       1
3       1
5       2
7       1
       ..
1015    2
1017    2
1018    2
1020    1
1021    1
Name: FullBath, Length: 526, dtype: int64
Missing values 0
Correlation to target            SalePrice
SalePrice   1.000000
FullBath    0.524085
Unique Values [1 2 3 0]


In [112]:
inspectContinous(numerical_columns[10], DF)
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'HalfBath')
droppedCont.append('HalfBath')

Name: HalfBath
1       0
2       0
3       1
5       1
7       1
       ..
1015    1
1017    1
1018    0
1020    1
1021    0
Name: HalfBath, Length: 526, dtype: int64
Missing values 0
Correlation to target            SalePrice
SalePrice    1.00000
HalfBath     0.18745
Unique Values [0 1 2]
Number of continuous features: 26
Number of categorical features: 0
Total Features: 32


In [113]:
inspectContinous(numerical_columns[10], DF)
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'BedroomAbvGr')
droppedCont.append('BedroomAbvGr')

Name: BedroomAbvGr
1       4
2       3
3       1
5       3
7       3
       ..
1015    3
1017    3
1018    3
1020    3
1021    3
Name: BedroomAbvGr, Length: 526, dtype: int64
Missing values 0
Correlation to target               SalePrice
SalePrice      1.000000
BedroomAbvGr   0.084986
Unique Values [4 3 1 2 5 0]
Number of continuous features: 25
Number of categorical features: 0
Total Features: 31


In [114]:
inspectContinous(numerical_columns[10], DF)
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'KitchenAbvGr')
droppedCont.append('KitchenAbvGr')

Name: KitchenAbvGr
1       1
2       1
3       1
5       1
7       1
       ..
1015    1
1017    1
1018    1
1020    1
1021    1
Name: KitchenAbvGr, Length: 526, dtype: int64
Missing values 0
Correlation to target               SalePrice
SalePrice      1.000000
KitchenAbvGr  -0.080116
Unique Values [1 2]
Number of continuous features: 24
Number of categorical features: 0
Total Features: 30


In [115]:
inspectContinous(numerical_columns[10], DF)

Name: TotRmsAbvGrd
1       7
2       6
3       4
5       7
7       6
       ..
1015    7
1017    7
1018    7
1020    6
1021    5
Name: TotRmsAbvGrd, Length: 526, dtype: int64
Missing values 0
Correlation to target               SalePrice
SalePrice      1.000000
TotRmsAbvGrd   0.540691
Unique Values [ 7  6  4  8  3  5 10  9 12 11]


In [116]:
inspectContinous(numerical_columns[11], DF)
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'Fireplaces')
droppedCont.append('Fireplaces')

Name: Fireplaces
1       1
2       1
3       1
5       1
7       1
       ..
1015    1
1017    1
1018    1
1020    1
1021    1
Name: Fireplaces, Length: 526, dtype: int64
Missing values 0
Correlation to target             SalePrice
SalePrice    1.000000
Fireplaces   0.157155
Unique Values [1 3 2]
Number of continuous features: 23
Number of categorical features: 0
Total Features: 29


In [117]:
inspectContinous(numerical_columns[11], DF)

Name: GarageCars
1       1
2       2
3       2
5       2
7       2
       ..
1015    2
1017    2
1018    2
1020    2
1021    1
Name: GarageCars, Length: 526, dtype: int64
Missing values 0
Correlation to target             SalePrice
SalePrice    1.000000
GarageCars   0.643213
Unique Values [1 2 3]


In [118]:
inspectContinous(numerical_columns[11], DF)

Name: GarageCars
1       1
2       2
3       2
5       2
7       2
       ..
1015    2
1017    2
1018    2
1020    2
1021    1
Name: GarageCars, Length: 526, dtype: int64
Missing values 0
Correlation to target             SalePrice
SalePrice    1.000000
GarageCars   0.643213
Unique Values [1 2 3]


In [119]:
inspectContinous(numerical_columns[12], DF)
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'WoodDeckSF')
droppedCont.append('WoodDeckSF')

Name: GarageArea
1       296
2       884
3       550
5       400
7       546
       ... 
1015    613
1017    440
1018    530
1020    576
1021    360
Name: GarageArea, Length: 526, dtype: int64
Missing values 0
Correlation to target             SalePrice
SalePrice     1.00000
GarageArea    0.60573
Unique Values [ 296  884  550  400  546  662  686  420  720  303  484  440  220  430
  300  349  836  516  492  678  393  600  834  275  500  312  336  850
  576  462  388  240  390  864  360  490  525  602  539  671  286  750
  765  566  451  890  447  180  590  234  748  540  264  270  672  288
  676  648  460  616  665  432  789  352  541  592  786  454  556  594
  636  564  418  528  714  680  439  403  856  527  343  506  410  434
  560  514  722  476  924 1020  450  402  308  338  225  205  431  445
  880  522  495  711  779  319  521  865  857  466  820  610  872  529
  304  213 1166  776  433  586  898  478  660  612  701  862  738  841
  534  583  373  216 1053  800  577  280  641  45

In [120]:
inspectContinous(numerical_columns[13], DF)
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'EnclosedPorch')
droppedCont.append('EnclosedPorch')

Name: OpenPorchSF
1        0
2        0
3       84
5       36
7       42
        ..
1015    39
1017     0
1018     0
1020     0
1021     0
Name: OpenPorchSF, Length: 526, dtype: int64
Missing values 0
Correlation to target              SalePrice
SalePrice     1.000000
OpenPorchSF   0.242514
Unique Values [  0  84  36  42  55  78  28 140  24  12  63 187  75  60  50 229  20  17
 118  30  57 111 234  56  38  40 150 130 120  90  72  51 213 138 418 174
  47  32  35 128  26  39  25  18 208 121 170  21  22   4  80 502  48  66
  76 258  16 228  59 123 241  10 214 154  85  74  49  70 312 185 159  15
 110  44 155  73 122 124 148 116 151  62 240  88 108  52 100 114 126 102
  81 134 112 231  65 168 263  98  61 104  33 132 175  45  69  53  29 119
  87  68  99 236 182 207 252 262  94  96 244 144 105 210 192 292 341  82
  67 184 211 243 204  58  54 235 180  43  64 162 189 136 137 133]
Number of continuous features: 21
Number of categorical features: 0
Total Features: 27


In [121]:
inspectContinous(numerical_columns[13], DF)
DF, categorical_columns, numerical_columns = updateDataFrame(DF, '3SsnPorch')
droppedCont.append('3SsnPorch')

Name: OpenPorchSF
1        0
2        0
3       84
5       36
7       42
        ..
1015    39
1017     0
1018     0
1020     0
1021     0
Name: OpenPorchSF, Length: 526, dtype: int64
Missing values 0
Correlation to target              SalePrice
SalePrice     1.000000
OpenPorchSF   0.242514
Unique Values [  0  84  36  42  55  78  28 140  24  12  63 187  75  60  50 229  20  17
 118  30  57 111 234  56  38  40 150 130 120  90  72  51 213 138 418 174
  47  32  35 128  26  39  25  18 208 121 170  21  22   4  80 502  48  66
  76 258  16 228  59 123 241  10 214 154  85  74  49  70 312 185 159  15
 110  44 155  73 122 124 148 116 151  62 240  88 108  52 100 114 126 102
  81 134 112 231  65 168 263  98  61 104  33 132 175  45  69  53  29 119
  87  68  99 236 182 207 252 262  94  96 244 144 105 210 192 292 341  82
  67 184 211 243 204  58  54 235 180  43  64 162 189 136 137 133]
Number of continuous features: 20
Number of categorical features: 0
Total Features: 26


In [122]:
inspectContinous(numerical_columns[13], DF)
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'ScreenPorch')
droppedCont.append('ScreenPorch')

Name: OpenPorchSF
1        0
2        0
3       84
5       36
7       42
        ..
1015    39
1017     0
1018     0
1020     0
1021     0
Name: OpenPorchSF, Length: 526, dtype: int64
Missing values 0
Correlation to target              SalePrice
SalePrice     1.000000
OpenPorchSF   0.242514
Unique Values [  0  84  36  42  55  78  28 140  24  12  63 187  75  60  50 229  20  17
 118  30  57 111 234  56  38  40 150 130 120  90  72  51 213 138 418 174
  47  32  35 128  26  39  25  18 208 121 170  21  22   4  80 502  48  66
  76 258  16 228  59 123 241  10 214 154  85  74  49  70 312 185 159  15
 110  44 155  73 122 124 148 116 151  62 240  88 108  52 100 114 126 102
  81 134 112 231  65 168 263  98  61 104  33 132 175  45  69  53  29 119
  87  68  99 236 182 207 252 262  94  96 244 144 105 210 192 292 341  82
  67 184 211 243 204  58  54 235 180  43  64 162 189 136 137 133]
Number of continuous features: 19
Number of categorical features: 0
Total Features: 25


In [123]:
inspectContinous(numerical_columns[13], DF)
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'PoolArea')
droppedCont.append('PoolArea')

Name: OpenPorchSF
1        0
2        0
3       84
5       36
7       42
        ..
1015    39
1017     0
1018     0
1020     0
1021     0
Name: OpenPorchSF, Length: 526, dtype: int64
Missing values 0
Correlation to target              SalePrice
SalePrice     1.000000
OpenPorchSF   0.242514
Unique Values [  0  84  36  42  55  78  28 140  24  12  63 187  75  60  50 229  20  17
 118  30  57 111 234  56  38  40 150 130 120  90  72  51 213 138 418 174
  47  32  35 128  26  39  25  18 208 121 170  21  22   4  80 502  48  66
  76 258  16 228  59 123 241  10 214 154  85  74  49  70 312 185 159  15
 110  44 155  73 122 124 148 116 151  62 240  88 108  52 100 114 126 102
  81 134 112 231  65 168 263  98  61 104  33 132 175  45  69  53  29 119
  87  68  99 236 182 207 252 262  94  96 244 144 105 210 192 292 341  82
  67 184 211 243 204  58  54 235 180  43  64 162 189 136 137 133]
Number of continuous features: 18
Number of categorical features: 0
Total Features: 24


In [124]:
inspectContinous(numerical_columns[13], DF)
DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'MiscVal')
droppedCont.append('MiscVal')

Name: OpenPorchSF
1        0
2        0
3       84
5       36
7       42
        ..
1015    39
1017     0
1018     0
1020     0
1021     0
Name: OpenPorchSF, Length: 526, dtype: int64
Missing values 0
Correlation to target              SalePrice
SalePrice     1.000000
OpenPorchSF   0.242514
Unique Values [  0  84  36  42  55  78  28 140  24  12  63 187  75  60  50 229  20  17
 118  30  57 111 234  56  38  40 150 130 120  90  72  51 213 138 418 174
  47  32  35 128  26  39  25  18 208 121 170  21  22   4  80 502  48  66
  76 258  16 228  59 123 241  10 214 154  85  74  49  70 312 185 159  15
 110  44 155  73 122 124 148 116 151  62 240  88 108  52 100 114 126 102
  81 134 112 231  65 168 263  98  61 104  33 132 175  45  69  53  29 119
  87  68  99 236 182 207 252 262  94  96 244 144 105 210 192 292 341  82
  67 184 211 243 204  58  54 235 180  43  64 162 189 136 137 133]
Number of continuous features: 17
Number of categorical features: 0
Total Features: 23


In [125]:
inspectContinous(numerical_columns[14], DF)

DF, categorical_columns, numerical_columns = updateDataFrame(DF, 'MoSold')
droppedCont.append('MoSold')

Name: MoSold
1       10
2        5
3        4
5        4
7        6
        ..
1015     1
1017     7
1018     5
1020    11
1021    12
Name: MoSold, Length: 526, dtype: int64
Missing values 0
Correlation to target            SalePrice
SalePrice   1.000000
MoSold      0.020012
Unique Values [10  5  4  6  7  1 11  3 12  2  8  9]
Number of continuous features: 16
Number of categorical features: 0
Total Features: 22


In [127]:
print(list(DF.columns))
print(droppedCols)
print(droppedCont)

['LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'TotalBsmtSF', '1stFlrSF', 'GrLivArea', 'FullBath', 'TotRmsAbvGrd', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'OpenPorchSF', 'YrSold', 'SalePrice', 'ExterQual_TA', 'Foundation_PConc', 'KitchenQual_TA']
['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual_Gd', 'ExterQual_Fa', 'ExterCond', 'Foundation_CBlock', 'Foundation_Slab', 'Foundation_Stone', 'Foundation_Wood', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']
['Unnamed: 0', 'Id'

Make sure to one hot encode the "continous" variables that i have and check my feature count. 